# MNIST Experiment: IID and Balanced Dataset

In [1]:
import torch
import torchvision

import numpy as np
import math

Loading in the data

In [2]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

Setting the hyperparameters

In [3]:
## Parameters:
n_epochs = 3
batch_size_train = 10000
batch_size_test = 500
log_interval = 500

In [4]:
train_loader = torch.utils.data.DataLoader(mnist_trainset,batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset,batch_size=batch_size_test, shuffle=False)

In [5]:
import papayaclient

Writing the model class. Here we use a simple two layer FC Net.

In [6]:
class TheModel(torch.nn.Module):

    def __init__(self):
        super(TheModel, self).__init__()

        self.linear1 = torch.nn.Linear(784, 10)
        #self.linear2 = torch.nn.Linear(400, 10)
        #self.relu = torch.nn.ReLU()

    def forward(self, x):
        x1 = x.flatten(start_dim = 1)
        return self.linear1(x1)
        #return self.linear2(self.relu(self.linear1(x1)))

Creating the clients using papaya

In [7]:
clients = []
for batchno, (ex_data, ex_labels) in enumerate(train_loader):
    clients.append(papayaclient.PapayaClient(dat = ex_data,
                                            labs = ex_labels,
                                            batch_sz = 500,
                                            num_partners = 5,
                                            model_class = TheModel,
                                            loss_fn = torch.nn.CrossEntropyLoss))

In [8]:
import random
random.shuffle(clients)

In [9]:
## Train the Nodes
num_epochs_total = 100
num_epochs_per_swap = 5
num_times = (num_epochs_total // num_epochs_per_swap)
for i in range(0, num_times):
    for n in clients:
        for j in range(0, num_epochs_per_swap):
            n.model_train_epoch()
    if i > 1 and i < num_times - 1 :
        for n in clients:
            n.select_partners(3)
        for n in clients:
            for i in range(0, 4) :
                n.update_partner_weights()
        n.average_partners()

In [10]:
for c in clients :
    print(c.logs['stringy'][99])

node3729epoch 99 loss 0.2394808977842331
node4822epoch 99 loss 0.21691952645778656
node1467epoch 99 loss 0.25188735127449036
node161epoch 99 loss 0.2410050332546234
node2576epoch 99 loss 0.3126402497291565
node4176epoch 99 loss 0.3378574252128601


In [11]:
accuracies = {}
with torch.no_grad():
    for i in clients :
        accuracies_node = []
        for batchno, (ex_data, ex_labels) in enumerate(test_loader) :
            accuracies_node.append(((i.model.forward(ex_data).argmax(dim = 1) == ex_labels).float().mean()).item())
        accuracies[i.node_id] = np.array(accuracies_node).mean()

In [12]:
accuracies

{3729: 0.9129999965429306,
 4822: 0.9082999974489212,
 1467: 0.9091999918222428,
 161: 0.9116000026464463,
 2576: 0.9113000065088273,
 4176: 0.9149999976158142}

Above we see the accuracy that the model at each node achieves on the held out test set.